# SYSC4906 Assignment 3

**Group Name:** Thao and Riley

**Student names:** Thao-Tran Le-Phuong and Riley MacKinnon

**Student numbers:** 100997443 and 100996542


# Discussion of Solution
Your notebook must begin with **this** text cell containing a description of your solution. In this discussion, include links to any resources that you used in developing your solution. Use proper MarkDown syntax to format your discussion.**This description should be approximately 500 words in length and cover the following:**

1. Which machine learning approach did you use?

2. How did you split your data between training and testing? 
_(e.g. hold-out test, cross-validation, repeated bootstrap samples, etc)_

3. How did you train your classifier?
If you used transfer learning, describe how you did so.

4. How did you estimate your future performance _(worst recall, best precision, overall accuracy)_?

   **Here are our final predictions:**
   1. Building with worst recall:
   2. Building with best precision
   3. Overall accuracy)

5. Discuss the performance of your model. Which buildings did it do the best/worst on and why? What are the strengths and limitations of your method. 

6. What would you have done differently if you had more time?


# Code to Train Your Method
_We will look at this, but will not run it when measuring your accuracy. Please structure your training code into logical steps, so that we can easily understand it_
## Step 1: Load the image dataset...

In [1]:
from glob import glob
from shutil import copy2

!git clone https://github.com/Thao-Tran/sysc4906

IMG_DIR = 'sysc4906/Assignment 3/Images/'
TRAIN_DIR = 'train'
VAL_DIR = 'validation'
TEST_DIR = 'test'
FOLDS = 5
buildings = ('AA','CB','CT','DT','FH','HP','HS','LB','MC','ME','ML','PA','RB','RO','SA','TB')

for building in buildings:
  building_glob = glob(IMG_DIR+building+'/*.jpg')
  training_size = int(0.85*len(building_glob))
  fold_size = training_size // FOLDS
  training_set = building_glob.copy()[:training_size]

  for fold in range(FOLDS):
    if fold < FOLDS - 1:
      validation_set = training_set.copy()[fold_size*fold:fold_size*(fold+1)]
      del training_set[fold_size*fold:fold_size*(fold+1)]
    else:
      validation_set = training_set.copy()[fold_size*fold:]
      del training_set[fold_size*fold:]
    
    %mkdir -p /content/{fold}/{TRAIN_DIR}/{building}
    %mkdir -p /content/{fold}/{VAL_DIR}/{building}
    for path in training_set:
      #%cp {path.replace(' ', '\ ')} /content/{fold}/{TRAIN_DIR}/{building}
      copy2(path, str(fold)+'/'+TRAIN_DIR+'/'+building)
    for path in validation_set:
      #%cp {path.replace(' ', '\ ')} /content/{fold}/{VAL_DIR}/{building}
      copy2(path, str(fold)+'/'+VAL_DIR+'/'+building)

  %mkdir -p {TEST_DIR}/{building}

  for path in building_glob[training_size:]:
    #%cp {path.replace(' ', '\ ')} {TEST_DIR}/{building}/
    copy2(path, TEST_DIR+'/'+building)

#rm -rf sysc4906

Cloning into 'sysc4906'...
remote: Enumerating objects: 3711, done.
remote: Total 3711 (delta 0), reused 0 (delta 0), pack-reused 3711
Receiving objects: 100% (3711/3711), 218.57 MiB | 40.92 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Checking out files: 100% (3647/3647), done.


##Step N: Save the model to file

In [2]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, Dropout

CLASSES = 16
WIDTH = 500
HEIGHT = 500
BATCH_SIZE = 32
EPOCHS = 10
STEPS_PER_EPOCH = 3
VALIDATION_STEPS = 1

for fold in range(FOLDS):
  base_model = InceptionV3(weights='imagenet', include_top=False)

  x = base_model.output
  x = GlobalAveragePooling2D(name='avg_pool')(x)
  x = Dropout(0.5)(x)
  predictions = Dense(CLASSES, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  for layer in base_model.layers:
      layer.trainable = False

  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  train_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

  validation_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

  train_generator = train_datagen.flow_from_directory(
      str(fold)+'/'+TRAIN_DIR,
      target_size=(WIDTH,HEIGHT),
      batch_size=BATCH_SIZE,
      class_mode='categorical')

  validation_generator = validation_datagen.flow_from_directory(
      str(fold)+'/'+VAL_DIR,
      target_size=(WIDTH,HEIGHT),
      batch_size=BATCH_SIZE,
      class_mode='categorical')

  history = model.fit_generator(
      train_generator,
      epochs=EPOCHS,
      steps_per_epoch=STEPS_PER_EPOCH,
      validation_data=validation_generator,
      validation_steps=VALIDATION_STEPS)
    
  model.save(str(fold)+'.model')

Using TensorFlow backend.















87916544/87910968 [==============================] - 1s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Found 2280 images belonging to 16 classes.
Found 565 images belonging to 16 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
3/3 [==============================] - 28s 9s/step - loss: 3.1101 - acc: 0.0521 - val_loss: 2.7726 - val_acc: 0.0625
Epoch 2/10
3/3 [==============================] - 4s 1s/step - loss: 2.9734 - acc: 0.0104 - val_loss: 2.9140 - val_acc: 0.0312
Epoch 3/10
3/3 [==============================] - 9s 3s/step - loss: 3.0742 - acc: 0.0521 - val_loss: 2.9595 - val_acc: 0.0625
Epoch 4/10
3/3 [==============================] - 11s 4s/step - loss: 3.0475 - acc: 0.0417 - val_loss: 2.7640 - val_acc: 0.0312
Epoch 5/10
3/3 [==============================] - 10s 3s/step - loss: 2.9524 - acc: 0.1146 - val_loss: 2.7890 - va

KeyboardInterrupt: ignored

#Required functions to test your method
_These are the five required methods that you must implement._

## prepareModel()
This function should prepare your model for multiple invocations of classifyImage(fname). For example, this function could be used to load a pre-trained model from a URL, where that model is then used by  classifyImage(fname). You should use global variables for any variables initialized by this function.

Runtime of this method is **limited to 5 minutes**, so please don’t retrain your network here. All training should be captured in a pre-trained model to be loaded by this method.


In [0]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import load_model

HEIGHT = 500
WIDTH = 500
FOLDS = 5

# List of building codes to use throughout notebook.
buildingCodes = ('AA', 'CB', 'CT', 'DT', 'FH', 'HP', 'HS', 'LB', 'MC', 'ME', 'ML', 'PA', 'RB', 'RO', 'SA', 'TB')

def prepareModel():
  global models # Sample global variable that you may populate here.
  models = []
  for fold in range(folds):
    models.append(load_model(str(fold)+'.model'))


## label = classifyImage(fname)
Accepts a filename (e.g. ‘test/ME/testImage1.jpg’) of a square JPG image with size at least 500x500 pixels.
Returns a 2-character label corresponding to the predicted building (see table of labels above)

Any variables initialized by prepareModel() should be declared as global within this function if you want to access them (e.g. a pre-trained model)


In [0]:
from keras.preprocessing import image as kp_image
from keras.applications.inception_v3 import preprocess_input
import numpy as np

def classifyImage(fname):
  global models # Sample global variable that you may populate in prepareModel and use here.

  print("Predicting class of '{0:s}' using model '{1:s}'".format(fname,model))
  img = kp_image.load_img(fname, target_size=(HEIGHT, WIDTH))
  x = kp_image.img_to_array(img)  # Convert image to nparray
  x = np.expand_dims(x, axis=0)   # Need to pre-pend a dimension to indicate batch number.
  x = preprocess_input(x)         # Normalize image to match how Inceptionv3 expects to receive images
  preds = np.zeros(len(buildingCodes), dtype=int)
  for model in models:
    preds += model.predict(x)[0]        # Use the model to compute prediction score for each possible class
  preds /= FOLDS
  pred = ''
  score = -1
  for i in range(len(preds)):
    if preds[i] > score:
      pred = buildingCodes[i]
      score = preds[i]
  return pred


In [0]:
from glob import glob
from sklearn.metrics import confusion_matrix, accuracy_score

TEST_DIR = 'test'

def get_metrics():
  y_true = []
  y_pred = []
  for building in buildingCodes:
    paths = glob(TEST_DIR+'/'+buildingCode+'/*.jpg')
    y_true += building * len(paths)
    y_pred += [classifyImage(path) for path in paths]
  return confusion_matrix(y_true, y_pred), accuracy_score(y_true, y_pred)

cm, accuracy = get_metrics()

## label = worstRecall()
Returns the label of a building that you expect will have to lowest recall, when tested on new images

In [0]:
from glob import glob

def worstRecall():
  worst_recall = -1
  building = ''
  for i in range(len(buildingCodes)):
    tn, fp, fn, tp = cm.ravel()
    recall = tp / (tp+fn)
    if worst_recall == -1 or recall < worst_recall:
      worst_recall = recall
      building = buildingCodes[i]
  return building


## label = bestPrecision()
Returns the label of a building that you expect will have to highest precision, when tested on new images

In [0]:
def bestPrecision():
  best_precision = -1
  building = ''
  for i in range(len(buildingCodes)):
    tn, fp, fn, tp = cm.ravel()
    precision = tp / (tp+fp)
    if precision > best_precision:
      best_precision = precision
      building = buildingCodes[i]
  return building


##acc_score = estimatedAccuracy()
Returns the accuracy (between [0.0,1.0]) that you expect to achieve across all test images, assuming that each building is equally represented


In [0]:
# Function to return estimated accuracy that will be obtained across all test images
def estimatedAccuracy():
  return accuracy

# Test required functions
_We will replace the text below with our actual test code..._

In [0]:
import numpy as np
from PIL import Image, ExifTags
import matplotlib.pyplot as plt
from glob import glob


# First prepare the model:
prepareModel() # Limited to 5 minutes...

# Load (secret) test data into local Colab environment
!wget https://github.com/jrgreen7/SYSC4906/blob/master/Assignments/Assignment3/Images/SampleImages.zip?raw=true
!unzip SampleImages.zip?raw=true #Should create 4 images
test_images = sorted(glob('SampleImages/*.jpg'))
actual_labels = (buildingCodes[0],buildingCodes[1],buildingCodes[3],buildingCodes[6])

# Classify sample test images:
TP = 0
for imgFname,actual_label in (zip(test_images,actual_labels)):
  pred_label = classifyImage(imgFname) # Predict the label of this image file

  # Plot the image with actual and predicted labels
  # Note that we may have to rotate the image, depending on the 
  # orientation of the camera. Use EXIF tags for this:
  im = Image.open(imgFname)
  for orientation in ExifTags.TAGS.keys() : 
    if ExifTags.TAGS[orientation]=='Orientation' : break 
  exif=dict(im._getexif().items())

  if exif[orientation] == 3 : 
    im=im.rotate(180, expand=True)
  elif exif[orientation] == 6 : 
    im=im.rotate(270, expand=True)
  elif exif[orientation] == 8 : 
    im=im.rotate(90, expand=True)

  im = im.convert('RGB') # May not be necessary?

  plt.title("Sample test image of {0:s} predicted as {1:s}".format(actual_label, pred_label))
  plt.imshow(np.asarray(im))
  plt.axis('off')
  plt.show()

  if (pred_label==actual_label):
    print('Correct!')
    TP += 1
  else:
    print("Incorrect...")

# Print the predicted performance:
print("Expected that worst recall would be on {0:s}".format(worstRecall()))
print("Expected that best precision would be on {0:s}".format(bestPrecision()))
print("Expected total accuracy would be {0:.3f}".format(estimatedAccuracy()))
print("Actual total accuracy is {0:.3f}".format((TP)/(len(test_images))))
